## 発表内容

最近日本語対応したAmazon KendraはAmazon S3やAmazon RDS、Salesforceといったリポジトリのコンテンツを一元管理されたインデックスに簡単に集約できます。これにより企業の様々なデータを素早く検索できます。そして、Amazon Comprehendはドキュメント内のテキストの解析を行うことができます。この2つのサービスを組み合わせた文書検索と検索された文書の解析についてデモを交えてご紹介します。

----

## 概要

- 非構造化データは増え続けている
- 非構造データから必要な情報を入手することが困難
- Kendraを使うと非構造データの検索自体は可能
- Kendraでジャンル等の絞り込みフィルタリングを行うことでより精度の高い検索実現したい・・・・

### 解決策（一例）

- Kendraではドキュメントのリストを絞り込むために、ドキュメント取り込み時にメタデータを割り当て、フィルタリング・ファセット検索ができる
- Comprehendを利用して自動でメタデータを生成する

### 結論

- 自動で非構造データの絞り込み検索ができちゃう！

----

## アーキテクチャ

![](./architecture_kendra_comprehend.png "アーキテクチャ図")

----
![](./kendra_flow.png "利用の流れ")

----

## デモの流れ

（このファイルと同じディレクトリにある事前準備.ipynbを先に実行しておきます、デモでは割愛します）

1. Comprehendのデモ
2. Wikipediaから取得したファイルをディレクトリに配置する（図①事前準備で実施済み）
3. Kendraインデックスとデータソースコネクタをプロビジョニングします（事前準備で実施済み）
4. Comprehendを用いたエンティティ抽出（図②）
5. S3へのデータ配置（図③）
6. データソースのsync（図④）
7. 検索！（図⑤）

## 1. Comprehendのデモ

https://console.aws.amazon.com/comprehend/v2/home?region=us-east-1#home

## 2. Wikipediaから取得したファイルをディレクトリに配置する（図①事前準備で実施済み）

ファイルリストは以下。

In [2]:
!ls -1 Data/

Amazon.com.html
Amazon_Elastic_Compute_Cloud.html
Amazon_Simple_Queue_Service.html
Amazon_Simple_Storage_Service.html
Amazon_Web_Services.html


## 3. Kendraインデックスとデータソースコネクタをプロビジョニングします（事前準備で実施済み）

新しい Kendraインデックス の作成には約15分ほどかかります。  
Cloudformationテンプレート の"./kendratemplate.yaml"で Kendraインデックス を作成しています。  

デモでは画面上でデータソースを作成する手順をご説明します。  

In [17]:
!sh provisioning.sh

comprehendlab-akazawt-20211117-22171
ComprehendDemo1117-001
{
    "StackId": "arn:aws:cloudformation:us-east-1:067150986393:stack/ComprehendDemo1117-001/9edcbcf0-478d-11ec-a8b3-0e6b8695f17f"
}


In [20]:
!aws cloudformation describe-stacks --stack-name "$(cat .stackname)" | grep StackStatus

            "StackStatus": "CREATE_COMPLETE",


## 4. Comprehendを用いたエンティティ抽出（図②）

事前に配置してあるファイルを対象として、Comprehendでエンティティ抽出を行う。この結果を検索時のメタデータとして利用する。  

ファイルを逐次読み込み -> Comprehendのdetect_entitiesメソッドでエンティティを取得 -> 整形してjson化してファイルに格納

```python
# ファイル読み込み
    doc_text = open(fname, 'r', encoding="utf-8").read()
...
# detect_entitiesメソッドでエンティティを取得
    for i in range(0, len(doc_text), compre_text_size):
        try:
            entities = client.detect_entities(Text=doc_text[i:i+compre_text_size], LanguageCode='ja')
...
# json化
    metadata["_source_uri"] = "https://ja.wikipedia.org/wiki/" + npreplace
    attributes["Attributes"] = metadata
    print(json.dumps(attributes, sort_keys=True, indent=4))
```

In [21]:
# 1ファイルでメタデータ抽出を試してみる
#  元のWikipediaページ
#    https://ja.wikipedia.org/wiki/Amazon_Web_Services

!python3 getpagemetadata.py Data/Amazon_Web_Services.html

{
    "Attributes": {
        "COMMERCIAL_ITEM": [
            "Route",
            "EC2",
            "Prime Video",
            "Kindle",
            "Echo",
            "Kindle Fire",
            "Fire",
            "Fire HD 10",
            "Fire TV",
            "Prime Air"
        ],
        "DATE": [
            "2月",
            "3月",
            "4月",
            "5月",
            "6月",
            "7月",
            "8月",
            "2006年",
            "2004年",
            "2021年9月2日"
        ],
        "EVENT": [],
        "LOCATION": [
            "アメリカ東部",
            "北バージニア",
            "アメリカ西部",
            "カリフォルニア",
            "サンパウロ",
            "アイルランド",
            "ロンドン",
            "シンガポール",
            "大阪",
            "香港"
        ],
        "ORGANIZATION": [
            "Amazon Web Services",
            "Wikipedia",
            "Amazon_Web_Services",
            "Amazon.com",
            "アマゾン",
            "AWS",
            "Amazon",
            "日本政府

In [22]:
# 一括で全ファイルのメタデータ抽出を実行する
!sh ./getmetadata.sh

Generate metadata for 5 pages
Starting from 0
Working on 0
Working on Amazon_Web_Services
Working on Amazon.com
Working on Amazon_Simple_Queue_Service
Working on Amazon_Elastic_Compute_Cloud
Working on Amazon_Simple_Storage_Service
Successful completion at 0


## 5. S3へのデータ配置（図③）

In [23]:
# 事前配置してあるテキスト(html)データと作成したエンティティメタデータをS3にアップロードする
!sh upload.sh

upload: Data/inference_pipeline_sparkml_xgboost_abalone.ipynb to s3://comprehendlab-akazawt-20211117-22171/Data/inference_pipeline_sparkml_xgboost_abalone.ipynb
upload: Data/Amazon_Web_Services.html to s3://comprehendlab-akazawt-20211117-22171/Data/Amazon_Web_Services.html
upload: Data/Amazon.com.html to s3://comprehendlab-akazawt-20211117-22171/Data/Amazon.com.html
upload: Data/Amazon_Simple_Queue_Service.html to s3://comprehendlab-akazawt-20211117-22171/Data/Amazon_Simple_Queue_Service.html
upload: Data/Amazon_Elastic_Compute_Cloud.html to s3://comprehendlab-akazawt-20211117-22171/Data/Amazon_Elastic_Compute_Cloud.html
upload: Data/Amazon_Simple_Storage_Service.html to s3://comprehendlab-akazawt-20211117-22171/Data/Amazon_Simple_Storage_Service.html
upload: Meta/Data/Amazon_Simple_Queue_Service.txt.metadata.json to s3://comprehendlab-akazawt-20211117-22171/Meta/Data/Amazon_Simple_Queue_Service.txt.metadata.json
upload: Meta/Data/Amazon_Simple_Storage_Service.txt.metadata.json to s3:/

## データソースのsync（図④）

現状、CloudFormationの[AWS::Kendra::DataSource](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/aws-properties-kendra-datasource-s3datasourceconfiguration.html#cfn-kendra-datasource-s3datasourceconfiguration-documentsmetadataconfiguration)でdefault languageに日本語を指定できないため、手動でデータソースの言語を日本語に設定する・・・

https://console.aws.amazon.com/kendra/home?region=us-east-1#indexes

In [25]:
!sh datasource_sync.sh

Kendra index ID is: 589a886f-aeec-40cd-9d9a-0b00415ce8a4. Kendra data source id is: e2ba2475-36ae-406b-a56d-b0a8303c3537
{
    "ExecutionId": "6f8e1d37-db99-41cd-8159-4986cfbdf63f"
}
{
    "History": [
        {
            "ExecutionId": "6f8e1d37-db99-41cd-8159-4986cfbdf63f",
            "StartTime": 1637144541.797,
            "Status": "SYNCING",
            "Metrics": {
                "DocumentsAdded": "0",
                "DocumentsModified": "0",
                "DocumentsDeleted": "0",
                "DocumentsFailed": "0",
                "DocumentsScanned": "0"
            }
        }
    ]
}
